In [ ]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!ls gdrive/My\ Drive/data

 data	        data_smaller.zip   old_data.zip   task3   task6
'data (1)'      data.zip	   task1	  task4   task7
 data_alfonso   full_data.zip	   task2	  task5


In [ ]:
!unzip gdrive/My\ Drive/data/data_smaller.zip > /dev/null

# THESE BOTTOM COMMAND IS FOR ALFONSO SINCE I HAD TO CREATE A SHORTCUT TO THE OG DATASET

In [ ]:
!ls gdrive/My\ Drive\/data_alfonso/data

ls: cannot access 'gdrive/My Drive/data_alfonso/data': No such file or directory


In [ ]:
!unzip  gdrive/My\ Drive\/data_alfonso/data/data.zip > /dev/nully

In [ ]:
# Unzip Dataset
"""import zipfile

with zipfile.ZipFile("./data.zip","r") as z:
    z.extractall(".")
    """

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split 
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder 
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob 
import random
import shutil
import cv2
import numpy as np

In [ ]:
# move in all of our collected data
task1_dataset = glob.glob(os.path.join('./data/task1/', '*.png'))
#task2_dataset = glob.glob(os.path.join('./data/task2/', '*.png'))
task2_dataset = glob.glob(os.path.join('./data/task2/', '*.png'))
#task4_dataset = glob.glob(os.path.join('./data/task4/', '*.png'))
#task5_dataset = glob.glob(os.path.join('./data/task5/', '*.png'))
#task6_dataset = glob.glob(os.path.join('./data/task6/', '*.png'))
#task7_dataset = glob.glob(os.path.join('./data/task7/', '*.png'))


if not os.path.exists("./training"):
  os.mkdir("./training")
  os.mkdir("./training/before")
  os.mkdir("./training/after")
if not os.path.exists("./validation"):
  os.mkdir("./validation")
  os.mkdir("./validation/before")
  os.mkdir("./validation/after")
if not os.path.exists("./test"): 
  os.mkdir("./test")
  os.mkdir("./test/before")
  os.mkdir("./test/after")

# this needs to be changed to 7 when we download data correctly
#tasks = [task1_dataset, task2_dataset, task3_dataset, task4_dataset, task5_dataset, task6_dataset, task7_dataset]
tasks = [task1_dataset, task2_dataset]
# now for each task randomly generate numbers for before and after to split
for i in range(2):
  current_task = tasks[i]
  training_size = int(len(current_task) * .8)
  test_size = int(len(current_task) * .1)
  val_size = int(len(current_task) * .1)

  # randomly choose an element
  generated_list = [j+1 for j in range(int(len(current_task) / 2))] 
  
  # add elements to our training array
  for k in range(int(training_size / 2)):
    random_index = random.randint(0,len(generated_list)-1)
    random_element = generated_list[random_index]

    # add the after and before images corresponding to this number 
    shutil.copy(f"./data/task{i+1}/before{random_element}.png", f"./training/before/task{i+1}_before_{random_element}.png")
    shutil.copy(f"./data/task{i+1}/after{random_element}.png", f"./training/after/task{i+1}_after_{random_element}.png")

    # remove that this element can be selected again
    del generated_list[random_index]


  # add elements to our test
  for k in range(int(test_size / 2)):

    random_index = random.randint(0,len(generated_list)-1)
    random_element = generated_list[random_index]

    # add the after and before images corresponding to this number 
    shutil.copy(f"./data/task{i+1}/before{random_element}.png", f"./test/before/task{i+1}_before_{random_element}.png")
    shutil.copy(f"./data/task{i+1}/after{random_element}.png", f"./test/after/task{i+1}_after_{random_element}.png")

    # remove that this element can be selected again
    del generated_list[random_index]

  
  # add elements to our validation
  for k in range(int(val_size / 2)):

    random_index = random.randint(0,len(generated_list)-1)
    random_element = generated_list[random_index]

    # add the after and before images corresponding to this number 
    shutil.copy(f"./data/task{i+1}/before{random_element}.png", f"./validation/before/task{i+1}_before_{random_element}.png")
    shutil.copy(f"./data/task{i+1}/after{random_element}.png", f"./validation/after/task{i+1}_after_{random_element}.png")

    # remove that this element can be selected again
    del generated_list[random_index]

  


In [ ]:
# Image transforms
dim = 512
def get_train_transform():
    return transforms.Compose([
    transforms.Resize((dim, dim)),
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0),(1, 1, 1))
    ])

def get_val_transform(): 
    return transforms.Compose([
    transforms.Resize((dim,dim)),
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0),(1, 1, 1))
    ])

In [ ]:
# PyTorch Dataset
# Source: https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
class CreateDataset(Dataset):
    
    def __init__(self, imgs, class_to_int, folder_type, transforms = None):
        super().__init__()
        self.imgs = imgs
        self.class_to_int = class_to_int
        self.transforms = transforms
        self.folder_type = folder_type
        
    def __getitem__(self, idx):
        image_name = self.imgs[idx]
        #print(image_name + "\n")
        indicators = image_name.split("_")
        task_num = indicators[0]
        image_num = indicators[2]
        
        before_dir = f"./{self.folder_type}/before/"
        before_img = self.transforms(Image.open(before_dir + image_name).convert('RGB'))
        after_dir = f"./{self.folder_type}/after/"
        image_name = task_num + "_after_" + image_num
        after_img = self.transforms(Image.open(after_dir + image_name).convert('RGB'))

        label = self.class_to_int[task_num]
        label = torch.tensor(label, dtype = torch.float32)

        image_list = [before_img, after_img]

        return image_list, label
            
        
    def __len__(self):
        return len(self.imgs)

In [ ]:
train_dir = "./training/before/"
train_images = os.listdir(train_dir) 

val_dir = "./validation/before/"
val_images = os.listdir(val_dir) 

test_dir = "./test/before/"
test_images = os.listdir(test_dir) 

In [ ]:
# Create Datasets to be used by Data Loaders
# class_mapping = {"task1" : 0, "task2" : 1, "task3" : 2, "task4": 3, "task5": 4, "task6": 5, "task7": 6}
class_mapping = {"task1" : 0, "task2" : 1}
train_data = CreateDataset(train_images, class_mapping, folder_type = "training", transforms = get_train_transform())
val_data = CreateDataset(val_images, class_mapping, folder_type = "validation", transforms = get_val_transform())
test_data = CreateDataset(test_images, class_mapping, folder_type = "test", transforms = get_val_transform())

In [ ]:
# Data Loaders
batch_size = 16
num_workers = 4
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
print('Num training images: ' + str(len(train_loader.dataset)))
print('Num validation images: ' + str(len(validation_loader.dataset)))
print('Num test images: ' + str(len(test_loader.dataset)))

Num training images: 104
Num validation images: 12
Num test images: 12


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Download ResNet50 from online to create our first network for before and the network for after

ResNet50_before = resnet50(weights=ResNet50_Weights.DEFAULT)
ResNet50_after = resnet50(weights=ResNet50_Weights.DEFAULT) 

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
ResNet50_before

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# CUT OFF THE LAYERS AT THE END

ResNet50_before.fc = nn.Sequential(
   nn.Linear(2048, 2048)
)

ResNet50_after.fc = nn.Sequential(
   nn.Linear(2048, 2048) 
)



In [ ]:
ResNet50_before

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# COMBINE BOTH MODELS INTO ONE CLASS
class ActionModel(nn.Module):
  def __init__(self):
    super(ActionModel, self).__init__()

    # NETWORK A IS OUR FIRST RESNET that takes in the before image
    self.model_before = ResNet50_before

    # NETWORK B IS OUR SECOND RESNET that takes in the after image
    self.model_after = ResNet50_after

    self.fc = nn.Sequential(
        nn.Linear(4096, 2048),
        nn.ReLU(),
        nn.Linear(2048, 512),
        nn.Linear(512, 2),
        nn.Softmax()
    )


  # forward propogation into our model
  def forward(self, before_image, after_image):

      # run forward prop on the before image
      before_encoded = self.model_before(before_image)

      # run forward prop on the after image
      after_encoded = self.model_after(after_image)

      # combine our encodings
      images_combined = torch.cat((before_encoded, after_encoded), 1)

      #run final layer of model and return the highest value
      return self.fc(images_combined)


In [ ]:
#Define Utility Functions 

#Accuracy function 
def accuracy(preds, labels):
  _, pred = torch.max(preds, dim=1)
  return torch.sum(pred==labels).item()/len(labels)

def check_model_accuracy(model, dloader):
    model.eval()
    acc = 0
    for batch, (X, y) in enumerate(dloader):
        #convert batch into variable that can have gradients applied to them
        X = X.to(device)
        y = y.to(device)
        #forward pass thru net
        preds = model(X[0], X[1])
        #check accuracy
        acc+=accuracy(preds, y)

    return acc/len(dloader)

In [ ]:
model = ActionModel()

In [ ]:
model

ActionModel(
  (model_before): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
 

In [ ]:
# TRAIN THE MODEL

#model = ActionModel()

#Freeze initial layers 

for i, child in enumerate(model.model_before.children()):
    if i<9:
        for param in child.parameters():
            param.requires_grad = False 
    else:
        for param in child.parameters():
            param.requires_grad = True

for i, child in enumerate(model.model_after.children()):
    if i<9:
        for param in child.parameters():
            param.requires_grad = False 
    else:
        for param in child.parameters():
            param.requires_grad = True

#init SGD optimizer & CrossEntropyLoss function
optim = torch.optim.SGD(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

#put on device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

n_epochs = 10
step=0

#Loop thru epochs 
for i in range(n_epochs):
    val_losses = []
    
    #loop thru samples
    for idx, (X, y) in enumerate(train_loader):
        
        X[0] = X[0].to(device)
        X[1] = X[1].to(device)
        mask = torch.unsqueeze(torch.all(torch.eq(X[0], X[1]),axis=1), axis = 1)
        print(X[0].shape)
        print(X[1].shape)
        print(mask.shape)

        mask = mask.repeat(1,3,1,1)
        print(mask.shape)

        print(X[0][0].shape)
        print(X[1][0].shape)
        print(mask[0].shape)

        result = np.concatenate((X[0][0].cpu().numpy(), X[1][0].cpu().numpy(), mask[0].cpu().numpy()), axis=2)

        plt.imsave("./training_images/epoch"+str(i)+"step"+str(idx)+".jpg", np.moveaxis(result,0,-1))
        

        y = y.to(device)
        
        #make predictions
        preds = model(X[0], X[1])
        
        #calculate loss 
        loss = loss_fn(preds.cpu(), y.cpu().to(torch.long))
        print(loss)
        #backprop
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        #check validation loss & accuracy 
        if step%100==0:
          #calculate accuracy 
            model.eval()
            val_acc=0
            val_loss=0

            with torch.no_grad():
                for batch, (X, y) in enumerate(validation_loader):

                    X[0] = X[0].to(device)
                    X[1] = X[1].to(device)
                    y = y.to(device)
                    preds = model(X[0],X[1])
                    
                    val_loss+=loss_fn(preds.cpu(), y.cpu().to(torch.long))
                    val_acc+=accuracy(preds.cpu(), y.cpu())
                
                #normalize val accuracy and loss
                val_acc = val_acc / len(validation_loader)
                val_loss = val_loss / len(validation_loader)
                    
                #early stopping
                if len(val_losses)>5:
                    if val_loss >= (sum(val_losses[i-5:i])/5):
                        print("Stopped at Epoch: ", idx)
                        break

                val_losses.append(val_loss)
        step+=1

    if len(val_losses) > 0:
      print("Epoch: ", i, " Val Loss: ", val_losses[-1], " Val Accuracy: ", val_acc)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([16, 3, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([16, 1, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(0.7001, grad_fn=<NllLossBackward0>)
torch.Size([16, 3, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([16, 1, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
tensor(0.6908, grad_fn=<NllLossBackward0>)
torch.Size([16, 3, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([16, 1, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
tensor(0.6945, grad_fn=<NllLossBackward0>)
torch.Size([16, 3, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([16, 1, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
tensor(0.6911, grad_fn=<NllLossBackward0>)
torch.Size([16, 3, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([16, 1, 512, 512])
torch.Size([16, 3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
tensor(0.6893, grad_fn=<NllLossBackw

KeyboardInterrupt: ignored

In [ ]:
# TEST THE MODEL

#Evaluate the accuracy on the test set.
check_model_accuracy(model, test_loader)

In [ ]:
#Unfreeze all layers.

for i, param in enumerate(model.parameters()):
    param.requires_grad = True 

#Continue Fine Tuning

#init SGD optimizer & CrossEntropyLoss function
optim = torch.optim.SGD(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

#put on device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

n_epochs = 10
step=0

#Loop thru epochs 
for i in range(n_epochs):
    val_losses = []
    
    #loop thru samples
    for idx, (X, y) in enumerate(train_loader):
        
        X = X.to(device)
        y = y.to(device)
        
        #make predictions
        preds = model(X[0], X[1])
        
        #calculate loss 
        loss = loss_fn(preds.cpu(), y.cpu())

        #backprop
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        #check validation loss & accuracy 
        if step%100==0:
          #calculate accuracy 
            model.eval()
            val_acc=0
            val_loss=0

            with torch.no_grad():
                for batch, (X, y) in enumerate(validation_loader):

                    X = X.to(device)
                    y = y.to(device)
                    preds = model(X[0],X[1])
                    
                    val_loss+=loss_fn(preds.cpu(), y.cpu())
                    val_acc+=accuracy(preds.cpu(), y.cpu())
                
                #normalize val accuracy and loss
                val_acc = val_acc / len(val_loader)
                val_loss = val_loss / len(val_loader)
                    
                #early stopping
                if len(val_losses)>5:
                    if val_loss >= (sum(val_losses[i-5:i])/5):
                        print("Stopped at Epoch: ", idx)
                        break

                val_losses.append(val_loss)
        step+=1
    print("Epoch: ", i, " Val Loss: ", val_losses[-1], " Val Accuracy: ", val_acc)

In [ ]:
# TEST THE MODEL

#Evaluate the accuracy on the test set.
check_model_accuracy(resnet, test_loader)